# 意思決定問題

## 問題設定

- 選択肢: 機械の稼働台数（0台・1台・2台）
- 自然の状態: 製品の需要量（好況・不況）

## パラメータ

- 工場の固定費用(fixed_cost): 100万円
- 機械1台の稼働コスト(run_cost): 600万円
- 製品1角つの販売価格(sale_price): 0.2万円
- 機械1台で作られる製品数(machine_ability): 5_000個
- 好況時の需要量(demand_boom): 100_000個
- 不況時の需要量(demand_slump): 5_000個

この時、製品製造数は以下となる。

$$\text{製品製造数}=\text{machine\_ability}\times\text{機械の稼働台数}$$

製品販売数は製品製造数と需要量の小さい方となる。

$$\text{製品販売数}=\min(\text{製品製造数},\text{需要量})$$

売上金額は製品販売数に販売価格をかけて算出できる。

$$\text{売上金額}=\text{製品販売数}\times\text{sale\_price}$$

製造コストは機械の稼働台数によって変化する。また、機会を1台も稼働させなくても固定費用が発生する。

$$\text{製造コスト}=\text{fixed\_cost}+\text{run\_cost}\times\text{機械の稼働台数}$$

売上金額から製造コストを引いたものが利得になる。

$$\text{利得}=\text{売上金額}-\text{製造コスト}$$


---
これらを踏まえてモデルを作成する。

In [1]:
import pandas as pd
import numpy as np

In [2]:
# パラメータ設定
fixed_cost = 100
run_cost = 600
sale_price = 0.2
machine_ability = 5_000
demand_boom = 10_000
demand_slump = 5_000

まずは利得行列を作成する。

In [4]:
num_product_df = pd.DataFrame({
    "0台": [0, 0],
    "1台": [min([machine_ability, demand_boom]),
        min([machine_ability, demand_slump]),
    ],
    "2台": [min([machine_ability * 2, demand_boom]),
        min([machine_ability * 2, demand_slump]),
    ]
})

num_product_df.index = ["好況", "不況"]

In [5]:
display(num_product_df)

,0台,1台,2台
好況,0,5000,10000
不況,0,5000,5000


これに製品の販売価格をかけると売り上げ行列が得られる。

In [6]:
sales_df = num_product_df * sale_price
display(sales_df)

,0台,1台,2台
好況,0.0,1000.0,2000.0
不況,0.0,1000.0,1000.0


次に製造コストを計算する。

In [7]:
run_cost_df = pd.DataFrame({
    "0台": np.repeat(fixed_cost, 2),
    "1台": np.repeat(fixed_cost + run_cost, 2),
    "2台": np.repeat(fixed_cost + run_cost * 2, 2),
})
run_cost_df.index = ["好況", "不況"]
display(run_cost_df)

,0台,1台,2台
好況,100,700,1300
不況,100,700,1300


売り上げ行列から製造コストを差し引いて利得行列を出す。

In [9]:
payoff_df = sales_df - run_cost_df
display(payoff_df)

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0
